In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import VarianceThreshold
from scipy.stats import pearsonr
import random
import tensorflow as tf
import pandas as pd
import csv
import math

In [4]:
file_path = "CM1-Modis.csv"
save_path = "QQ_CM1-Modis.csv"

In [5]:
def read_csv_as_list(file_path):
    data = []
    with open(file_path, 'r', newline='', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        first_row = True
        for row in csv_reader:
            if first_row:  
                first_row = False
                continue
            data.append(row)
    return data

In [6]:
csv_data = read_csv_as_list(file_path)

In [7]:
print(csv_data)

[['  The DPU-BOOT CSC shall include a DRAM BIT consisting of two write/read/compare tests.  The first test shall write the address of each memory location to that location.  The second test shall write the complement of each memory location to that location.', ' Startup SequenceThe DPU FSW is booted using PROM-resident bootstrap software.  The bootstrap software performs a basic set of built-in tests, then copies the DPU FSW from EEPROM to DRAM and executes it.  In flight, there are two methods which can trigger the DPU FSW to boot:* power-on (cold boot), or* watchdog reset (warm boot, commanded reboot).', '1'], ['  The DPU-BOOT CSC shall include a DRAM BIT consisting of two write/read/compare tests.  The first test shall write the address of each memory location to that location.  The second test shall write the complement of each memory location to that location.', 'initial boot sequencethe initial boot sequence, identified as Boot Sequence 1.  This processing takes place when the Bo

In [8]:
len(csv_data)

965

In [9]:
high_column = list({row[0] for row in csv_data})
low_column = list({row[1] for row in csv_data})

In [10]:
len(high_column)

167

In [11]:
len(low_column)

170

In [14]:
def do_query_terms(documents):
    stopwords_file_path = "stopwords.txt"
    with open(stopwords_file_path, 'r', encoding='utf-8') as file:
        stop_words = file.read().splitlines()
    doc = ' '.join(documents)
    words = doc.lower().split()
#     words = documents.lower().split()
    filtered_words = [word for word in words if word not in stop_words]
    query_terms = list(set(filtered_words))
    return query_terms

In [15]:
import math
import statistics

# idf
def calculate_idf(document, terms):
    
    idf_values = {}
    total_documents = len(documents)
    terms = do_query_terms(documents)
    for term in terms:
        
        doc_count = sum(1 for doc in documents if term in doc)
        
        idf_values[term] = math.log(total_documents / (1 + doc_count))  
    return idf_values


In [16]:

def average_idf_in_text(text, documents):
    

    text_terms = do_query_terms(text)
    
    idf_values = calculate_idf(text, text_terms)
   
    idf_sum = sum(idf_values.values())
    
    avg_idf = idf_sum / len(text_terms)
    return avg_idf

In [17]:
def max_idf_in_query(text, documents):
    
    query_terms = do_query_terms(text)
   
    idf_values = calculate_idf(query, query_terms)
   
    max_idf = max(idf_values.values())
    return max_idf


def stddev_idf_in_query(text, documents):
   
    query_terms = do_query_terms(text)
     
    idf_values = calculate_idf(query, query_terms)
    
    idf_list = list(idf_values.values())
    
    stddev_idf = statistics.stdev(idf_list)
    return stddev_idf

In [18]:
# documents = ["This is a document.", "Another document here.", "Yet another document."]
# query = "This is a new document."
# max_idf = max_idf_in_query(query, documents)
# stddev_idf = stddev_idf_in_query(query, documents)
# avg_idf = average_idf_in_text(query, documents)
# print("Average IDF in the query:", avg_idf)
# print("Max IDF in the query:", max_idf)
# print("Standard deviation of IDF in the query:", stddev_idf)

Average IDF in the query: 0.0090602335120295
Max IDF in the query: 0.4054651081081644
Standard deviation of IDF in the query: 0.3482374531411765


In [19]:
import math
import statistics


def calculate_ictf(document, terms):
    
    ictf_values = {}
    total_documents = len(documents)
    for term in terms:
       
        term_count = sum(1 for doc in documents if term in doc)
        
        if term_count != 0:
            
            ictf_values[term] = math.log(total_documents / term_count)  
        else:
            
            ictf_values[term] = 0
    return ictf_values


def average_ictf_in_query(query, documents):
    
    query_terms = set(query.split())
    
    ictf_values = calculate_ictf(query, query_terms)
    
    ictf_sum = sum(ictf_values.values())
    
    avg_ictf = ictf_sum / len(query_terms)
    return avg_ictf

# MaxICTF
def max_ictf_in_query(query, documents):
    
    query_terms = set(query.split())
    
    ictf_values = calculate_ictf(query, query_terms)
    
    max_ictf = max(ictf_values.values())
    return max_ictf

# DevICTF
def stddev_ictf_in_query(query, documents):
    
    query_terms = set(query.split())
    
    ictf_values = calculate_ictf(query, query_terms)
    
    ictf_list = list(ictf_values.values())
    
    stddev_ictf = statistics.stdev(ictf_list)
    return stddev_ictf

# 使用示例
documents = ["This is a document.", "Another document here.", "Yet another document."]
query = "This is a new document."
avg_ictf = average_ictf_in_query(query, documents)
max_ictf = max_ictf_in_query(query, documents)
stddev_ictf = stddev_ictf_in_query(query, documents)
print("Average ICTF in the query:", avg_ictf)
print("Max ICTF in the query:", max_ictf)
print("Standard deviation of ICTF in the query:", stddev_ictf)


Average ICTF in the query: 0.6016309587105096
Max ICTF in the query: 1.0986122886681098
Standard deviation of ICTF in the query: 0.48293446165521786


In [20]:
import math
import statistics

# entropy
def calculate_entropy(term, documents):
    
    tf_d_t_list = []
    total_documents = len(documents)
    for doc in documents:
        tf_d_t = doc.count(term) / len(doc.split()) 
        tf_t_D = sum(1 for doc in documents if term in doc) / total_documents 
       
        if tf_t_D != 0:
            entropy_term = (tf_d_t / tf_t_D) * math.log((tf_d_t / tf_t_D) + 1e-10) 
            tf_d_t_list.append(entropy_term)
    entropy = -sum(tf_d_t_list)
    return entropy

# AvgEntropy
def average_entropy_in_query(query, documents):
    
    query_terms = set(query.split())
    
    entropy_values = [calculate_entropy(term, documents) for term in query_terms]
    
    avg_entropy = sum(entropy_values) / len(entropy_values)  
    return avg_entropy


def median_entropy_in_query(query, documents):
    
    query_terms = set(query.split())
    
    entropy_values = [calculate_entropy(term, documents) for term in query_terms]
    
    med_entropy = statistics.median(entropy_values)
    return med_entropy


def max_entropy_in_query(query, documents):
    
    query_terms = set(query.split())
    
    entropy_values = [calculate_entropy(term, documents) for term in query_terms]
    
    max_entropy = max(entropy_values)
    return max_entropy


def stddev_entropy_in_query(query, documents):
    
    query_terms = set(query.split())
    
    entropy_values = [calculate_entropy(term, documents) for term in query_terms]
    
    stddev_entropy = statistics.stdev(entropy_values)
    return stddev_entropy


Average Entropy in the query: 0.20726660237906583
Median Entropy in the query: 0.21576155423883567
Max Entropy in the query: 0.7143845599593699
Standard deviation of Entropy in the query: 0.5527998782056432


In [21]:
import math

# Query Scope (QS)
def query_scope(query, documents):
    query_terms = set(query.split())
    containing_docs = [doc for doc in documents if any(term in doc for term in query_terms)]
    qs = len(containing_docs) / len(documents)
    return qs

# Simplified Clarity Score (SCS)
def simplified_clarity_score(query, documents):
    query_terms = set(query.split())
    query_model = {term: query.count(term) / len(query.split()) for term in query_terms}
    collection_model = {}
    total_terms = 0
    for doc in documents:
        for term in doc.split():
            total_terms += 1
            if term in collection_model:
                collection_model[term] += 1
            else:
                collection_model[term] = 1
    for term in collection_model:
        collection_model[term] /= total_terms

    scs = sum(query_model[term] * math.log(query_model[term] / collection_model.get(term, 1e-10) + 1e-10) for term in query_terms)
    return scs

Query Scope (QS): 0.6666666666666666
Simplified Clarity Score (SCS): 5.115059220223206


In [22]:
import math


def calculate_var(term, documents):
    total_documents = len(documents)
    term_count = sum(1 for doc in documents if term in doc)
    if term_count == 0:
        return 0  
    idf = math.log(total_documents / (1 + term_count))
    w_t_d_list = []
    for doc in documents:
        tf_t_d = doc.count(term) / len(doc.split())
        w_t_d = (1 / len(doc.split())) * math.log(1 + tf_t_d) * idf
        w_t_d_list.append(w_t_d)
    mean_w_t = sum(w_t_d_list) / term_count
    var = sum((w_t_d - mean_w_t) ** 2 for w_t_d in w_t_d_list) / term_count
    return math.sqrt(var)

#  AvgVAR
def avg_var(query, documents):
    query_terms = set(query.split())
    var_values = [calculate_var(term, documents) for term in query_terms]
    avg_var = sum(var_values) / len(var_values)
    return avg_var

#  MaxVAR
def max_var(query, documents):
    query_terms = set(query.split())
    var_values = [calculate_var(term, documents) for term in query_terms]
    max_var = max(var_values)
    return max_var

#  SumVAR
def sum_var(query, documents):
    query_terms = set(query.split())
    var_values = [calculate_var(term, documents) for term in query_terms]
    sum_var = sum(var_values)
    return sum_var


Average VAR: 0.01802265830506719
Max VAR: 0.058124868219087696
Sum VAR: 0.09011329152533595


In [24]:
import math

def calculate_pmi(term1, term2, documents):
    term1_count = sum(term1 in doc for doc in documents)
    term2_count = sum(term2 in doc for doc in documents)
    co_occurrence_count = sum(term1 in doc and term2 in doc for doc in documents)
    
    total_docs = len(documents)
    p_t1_D = term1_count / total_docs
    p_t2_D = term2_count / total_docs
    p_t1_t2_D = co_occurrence_count / total_docs
    
    if p_t1_D == 0 or p_t2_D == 0 or p_t1_t2_D == 0:
        return float('-inf')  
    pmi = math.log(p_t1_t2_D / (p_t1_D * p_t2_D))
    return pmi

def avg_pmi(query, documents):
    query_terms = query.split()
    pmi_values = []
    for term1 in query_terms:
        for term2 in query_terms:
            if term1 != term2:  
                pmi = calculate_pmi(term1, term2, documents)
                if pmi != float('-inf'):
                    pmi_values.append(pmi)
    # PMI 值
    if not pmi_values:
        return float('-inf')  
    avg_pmi_value = sum(pmi_values) / len(pmi_values)
    return avg_pmi_value

#  MaxPMI
def max_pmi(query, documents):
    query_terms = set(query.split())
    max_pmi = float('-inf')
    for term1 in query_terms:
        for term2 in query_terms:
            if term1 != term2:  # 避免重复计算同一对术语的PMI
                pmi = calculate_pmi(term1, term2, documents)
                max_pmi = max(max_pmi, pmi)
    return max_pmi

In [25]:
# 计算相似度函数
def calculate_similarity(doc1, doc2):
    # Placeholder for similarity calculation
    # Here you can implement any similarity measure, such as cosine similarity or Jaccard similarity
    # For demonstration, let's assume a simple similarity measure based on the number of common words
    words1 = set(doc1.split())
    words2 = set(doc2.split())
    common_words = words1.intersection(words2)
    similarity = len(common_words) / (len(words1) + len(words2))
    return similarity

# 计算 Coherence Score (CS)
def coherence_score(query, documents):
    query_terms = set(query.split())
    cs_values = []
    for term in query_terms:
        term_docs = [doc for doc in documents if term in doc]
        num_docs = len(term_docs)
        if num_docs < 2:  # If only one or zero document contains the term, skip
            continue
        sim_sum = 0
        for i in range(num_docs):
            for j in range(i + 1, num_docs):
                sim_sum += calculate_similarity(term_docs[i], term_docs[j])
        cs_values.append(sim_sum / (num_docs * (num_docs - 1)))
    if not cs_values:
        return 0  # If no query term has multiple documents, return 0
    coherence_score = sum(cs_values) / len(cs_values)
    return coherence_score

# 使用示例
documents = ["This is a document.", "Another document here.", "Yet another document."]
query = "This is a new document."
cs_value = coherence_score(query, documents)
print("Coherence Score (CS):", cs_value)


Coherence Score (CS): 0.07142857142857142


In [26]:
def calculate_metrics(query, documents):
    # 计算各种指标值
    max_idf = max_idf_in_query(query, documents)
    stddev_idf = stddev_idf_in_query(query, documents)
    avg_idf = average_idf_in_text(query, documents)
    avg_ictf = average_ictf_in_query(query, documents)
    max_ictf = max_ictf_in_query(query, documents)
    stddev_ictf = stddev_ictf_in_query(query, documents)
    avg_entropy = average_entropy_in_query(query, documents)
    med_entropy = median_entropy_in_query(query, documents)
    max_entropy = max_entropy_in_query(query, documents)
    dev_entropy = stddev_entropy_in_query(query, documents)
    qs = query_scope(query, documents)
    scs = simplified_clarity_score(query, documents)
    avg_var_value = avg_var(query, documents)
    max_var_value = max_var(query, documents)
    sum_var_value = sum_var(query, documents)
    cs_value = coherence_score(query, documents)
    avg_pmi_value = avg_pmi(query, documents)
    max_pmi_value = max_pmi(query, documents)
    
    # 返回指标值的列表
    metrics = [
        max_idf, stddev_idf, avg_idf, 
        avg_ictf, max_ictf, stddev_ictf,
        avg_entropy, med_entropy, max_entropy, dev_entropy,
        qs, scs,
        avg_var_value, max_var_value, sum_var_value,
        cs_value,
        avg_pmi_value, max_pmi_value
    ]
    return metrics

# # 使用示例
# documents = ["This is a document.", "Another document here.", "Yet another document."]
# query = "This is a new document."
# metrics = calculate_metrics(query, documents)
# print(metrics)


In [27]:
# len(metrics)

In [28]:
# csv_data[0][0]
len(csv_data)

965

In [29]:
high_dataset = []
low_dataset = []
for i in range(0,len(csv_data)):
#     query = csv_data[i][0]
#     documents = low_column
    high_dataset.append(calculate_metrics(csv_data[i][0], low_column))
    low_dataset.append(calculate_metrics(csv_data[i][1], high_column))
    

In [30]:
dataset = []
for i in range(len(csv_data)):
    combined_row = csv_data[i][:2] + high_dataset[i] + low_dataset[i] + [csv_data[i][2]]
    dataset.append(combined_row)

In [31]:
dataset

[['  The DPU-BOOT CSC shall include a DRAM BIT consisting of two write/read/compare tests.  The first test shall write the address of each memory location to that location.  The second test shall write the complement of each memory location to that location.',
  ' Startup SequenceThe DPU FSW is booted using PROM-resident bootstrap software.  The bootstrap software performs a basic set of built-in tests, then copies the DPU FSW from EEPROM to DRAM and executes it.  In flight, there are two methods which can trigger the DPU FSW to boot:* power-on (cold boot), or* watchdog reset (warm boot, commanded reboot).',
  0.4054651081081644,
  0.3482374531411765,
  0.005353774348017431,
  0.03118962370062803,
  0.4054651081081644,
  0.11018371921387925,
  7.339167874887698,
  2.9181933632110737,
  55.731503299372946,
  13.14397137330728,
  1.0,
  6.146267605148213,
  0.004991836459569646,
  0.023582646325147408,
  0.1297877479488108,
  0.042770623560757476,
  0.4098282311656275,
  3.34403896782220

In [32]:
df = pd.DataFrame(dataset)

In [33]:
df

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
0,The DPU-BOOT CSC shall include a DRAM BIT co...,Startup SequenceThe DPU FSW is booted using P...,0.405465,0.348237,0.005354,0.031190,0.405465,0.110184,7.339168,2.918193,...,13.682576,1.000000,10.495263,0.019061,0.191683,0.934004,0.059198,0.260423,4.424847,1
1,The DPU-BOOT CSC shall include a DRAM BIT co...,initial boot sequencethe initial boot sequence...,0.405465,0.348237,0.005354,0.031190,0.405465,0.110184,7.339168,2.918193,...,10.654688,1.000000,11.740890,0.015944,0.126291,1.243605,0.067394,0.190360,3.731699,1
2,The DPU-BOOT CSC shall include a DRAM BIT co...,DRAM BITPart of the memory is tested in Boot S...,0.405465,0.348237,0.005354,0.031190,0.405465,0.110184,7.339168,2.918193,...,11.398444,1.000000,10.182262,0.022858,0.245290,2.582910,0.072380,0.261472,5.117994,1
3,The DPU-BOOT CSC shall create a record of fa...,initial boot sequencethe initial boot sequence...,0.405465,0.348237,0.005354,0.054062,0.405465,0.142669,11.517246,2.992634,...,10.654688,1.000000,11.740890,0.015944,0.126291,1.243605,0.067394,0.190360,3.731699,1
4,The DPU-BOOT CSC shall create a record of fa...,DRAM BITPart of the memory is tested in Boot S...,0.405465,0.348237,0.005354,0.054062,0.405465,0.142669,11.517246,2.992634,...,11.398444,1.000000,10.182262,0.022858,0.245290,2.582910,0.072380,0.261472,5.117994,1
5,The DPU-BOOT CSC shall create a record of fa...,Exclude Failed DRAMThe Bootstrap performs a me...,0.405465,0.348237,0.005354,0.054062,0.405465,0.142669,11.517246,2.992634,...,14.451905,1.000000,7.488477,0.039696,0.667402,3.413893,0.064694,0.348747,5.117994,1
6,The DPU-BOOT CSC shall create a record of fa...,Built-In Test CSCThe Built-In Test (BIT) CSC i...,0.405465,0.348237,0.005354,0.054062,0.405465,0.142669,11.517246,2.992634,...,13.246786,1.000000,8.127646,0.028369,0.249721,1.957430,0.066931,0.226111,4.424847,1
7,The DPU-BOOT CSC shall create a record of fa...,External Program InterfacesReads the BIT resul...,0.405465,0.348237,0.005354,0.054062,0.405465,0.142669,11.517246,2.992634,...,13.603952,0.982036,8.157415,0.022150,0.216014,0.487306,0.054730,0.144822,2.345405,1
8,The DPU-BOOT CSC shall provide a monitor whi...,Bootstrap MonitorThe Bootstrap Monitor checks ...,0.405465,0.348237,0.005354,0.057924,0.405465,0.147239,9.118301,2.209984,...,13.629422,1.000000,6.991414,0.025621,0.191683,0.922350,0.076923,0.599952,4.424847,1
9,The Bootstrap Monitor of the DPU-BOOT CSC sh...,Built-In TestThere are two stages of Built-In ...,0.405465,0.348237,0.004908,0.015017,0.405465,0.078032,5.721020,0.601920,...,16.186538,1.000000,7.763545,0.052212,0.667402,2.192919,0.094815,0.354860,4.424847,1


In [34]:
# save_path

In [35]:
df.to_csv(save_path, index=False, encoding='utf-8')